In [4]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Json"
  // #r "nuget: Plotly.NET.ImageExport"

open Plotly.NET
// open Plotly.NET.ImageExport  

#load "Helpers.fs"
#load "Bestiary.fs"
#load "Compare.fs"
#load "Library.fs"
#load "Transform.fs"

open PathfinderAnalysis.Library
open PathfinderAnalysis.Bestiary
open PathfinderAnalysis.Compare
open PathfinderAnalysis.Transform
open PathfinderAnalysis.Helpers

Installed Packages FSharp.Json, 0.4.1 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\emmer\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [ ]:
let titleFn level = 
  sprintf "Middle Save - Level %i - Spout Cantrip vs. Martial Bow Against Level %i Creatures" level (level+2)

let mapShortbowAverages =
  transformedResultsByRollByLevel martialShortbow PlayerAttack creatureAc ((+) -5 << highMartialAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

let shortbowAverages =
  transformedResultsByRollByLevel martialShortbow PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel spout CreatureSave middleSave (casterDc true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Spout"
  };
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel telekineticProjectile PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Telekinetic Projectile"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Arbalest (1 shot)"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel shortbowAverages mapShortbowAverages
      |> Seq.toArray;
    Title = "Shortbow (2 shots)"
  }
|]
|> flatten
|> generateCharts titleFn
// |> Seq.iteri (fun i chart -> Chart.savePNG (path = (sprintf "spout-arbalest-%02i" i)) chart)
// |> ignore


In [ ]:
let titleFn level = 
  sprintf "Middle Save - Level %i - Caster Focus Spells vs. Fighter Weapons Against Level %i Creatures" level (level+2)

let mapShortbowAverages =
  transformedResultsByRollByLevel fighterShortbow PlayerAttack creatureAc ((+) -5 << highFighterAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

let shortbowAverages =
  transformedResultsByRollByLevel fighterShortbow PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

let doubleShotAverages =
  transformedResultsByRollByLevel fighterShortbow PlayerAttack creatureAc ((+) -2 << highFighterAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel tempestSurge CreatureSave middleSave (casterDc true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Tempest Surge"
  };
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel fireRay PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Fire Ray";
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel fighterArbalest PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Arbalest (1 shot)"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel shortbowAverages mapShortbowAverages
      |> Seq.toArray;
    Title = "Shortbow (2 shots)"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel doubleShotAverages doubleShotAverages
      |> Seq.toArray;
    Title = "Shortbow (Double Shot)"
  }
|]
|> flatten
|> generateCharts titleFn

In [ ]:
let monsterLevelDiff = 4

let titleFn level = 
  sprintf "Middle Save - Level %i - Caster Single-Target vs. Fighter Weapons Against Level %i Creatures" level (level+monsterLevelDiff)

let longswordAverage =
  transformedResultsByRollByLevel fighterLongsword PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
  |> Seq.map resultRollsToAverages

let shortswordAverage =
  transformedResultsByRollByLevel fighterShortsword PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel thunderstrike CreatureSave middleSave (casterDc true) bestiaryByLevel monsterLevelDiff
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Thunderstrike"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel longswordAverage shortswordAverage
      |> Seq.toArray;
    Title = "Double Slice"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel (forceBarrage 3) PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
    |> Seq.map resultRollsToAverages
    |> Seq.toArray;
    Title = "3-action force barrage"
  }
|]
|> flatten
|> generateCharts titleFn

In [35]:
#load "DamageDistribution.fs"

open PathfinderAnalysis.DamageDistribution

let die = D6
let rolls = 12
let nBuckets = 20

let average =
  float rolls * averageRoll die

let values = 
  rollDistribution die rolls
  |> chunk nBuckets
  |> chunksToAverages

let title = sprintf "Roll Sum of %i%s (average: %.1f) mapped onto a d%i" rolls (rollNotation die) average nBuckets

Chart.Line(
  x = List.map first values,
  y = List.map second values,
  ShowMarkers = true,
  MultiText = (List.map second values |> List.map (sprintf "%.0f")),
  TextPosition = StyleParam.TextPosition.TopCenter
)
|> Chart.withXAxisStyle(TitleText = "D20 Result")
|> Chart.withYAxisStyle(TitleText = "Estimated Roll Sum")
|> Chart.withTitle(title)



<!-- Plotly chart will be drawn inside this DIV -->